In [363]:
import os
import pandas as pd
import soccerdata as sd

In [364]:
# Primero Sofascore
folder1 = "Sofascore"
ligas = ["Premier","Ligue_1","Bundesliga"]
años = ["19-20", "20-21", "21-22", "22-23", "23-24"]
torneos = []

for liga in ligas:
    for año in años:
        match_file = f"stats_{año}.csv"
        try:
            path_matches = os.path.join(folder1, liga, match_file)
            matches = pd.read_csv(path_matches)
        except FileNotFoundError:
            continue
        torneos.append(matches)
df_sofascore = pd.concat(torneos).add_prefix("Sofascore_")

In [365]:
sd.MatchHistory.available_leagues()

['ARG-PrimeraDivision',
 'BRA-Brasileirao',
 'CHI-PrimeraDivision',
 'ENG-Premier League',
 'ESP-La Liga',
 'FRA-Ligue 1',
 'GER-Bundesliga',
 'ITA-Serie A',
 'NED-Eredivisie',
 'RUS-PremierLeague',
 'SCO-Premiership',
 'TUR-SuperLig',
 'USA-MLS']

In [366]:
folder2 = "FootballData"
ligas = [
    'ENG-Premier League',
    'ESP-La Liga',
    'FRA-Ligue 1',
    'GER-Bundesliga',
    'ITA-Serie A']
torneos = []
for liga in ligas:
    match_file = f"FootballData_{liga}.csv"
    try:
        path_matches = os.path.join(folder2, match_file)
        matches = pd.read_csv(path_matches)
    except FileNotFoundError:
        continue
    torneos.append(matches)

df_FootballData = pd.concat(torneos)

df_FootballData = df_FootballData.drop(
    ['Unnamed: 0.1'], axis=1).add_prefix("FootballData_")

# Convertir la columna 'Fecha' a tipo datetime
df_FootballData['FootballData_Date'] = pd.to_datetime(df_FootballData['FootballData_Date'], format='%d/%m/%Y')

# Convertir la zona horaria a UTC
df_FootballData['FootballData_Date'] = df_FootballData['FootballData_Date'].dt.tz_localize('UTC')

In [367]:
folder3 = "ESPN"
ligas = [
    'ENG-Premier League',
    'ESP-La Liga',
    'FRA-Ligue 1',
    'GER-Bundesliga',
    'ITA-Serie A']
torneos = []
for liga in ligas:
    match_file = f"ESPN_{liga}.csv"
    try:
        path_matches = os.path.join(folder3, match_file)
        matches = pd.read_csv(path_matches)
        matches = matches.drop("roster",axis=1)
    except FileNotFoundError:
        continue
    torneos.append(matches)
df_ESPN = pd.concat(torneos)
df_ESPN = df_ESPN.drop(["season","Unnamed: 0"], axis = 1)
df_ESPN = df_ESPN.drop_duplicates()


In [311]:
games = df_ESPN["game"].unique()
resultados = []
for i in  tqdm(range(len(games))):
    aux = df_ESPN[df_ESPN["game"]==games[i]]
    away = aux[aux["is_home"] == False].add_prefix("away_")
    home = aux[aux["is_home"] == True].add_prefix("home_")
    resultados.append(pd.DataFrame(pd.concat([home.iloc[0],away.iloc[0]])).T)

100%|██████████| 35761/35761 [02:06<00:00, 282.60it/s]


In [368]:
df_ESPN = pd.concat(resultados).add_prefix("ESPN_")

In [369]:
df_ESPN[['Fecha', 'Estadio']] = df_ESPN['ESPN_home_game'].str.split(' ', n=1, expand=True)
# Convertir la columna 'Fecha' a tipo datetime
df_ESPN['Fecha'] = pd.to_datetime(df_ESPN['Fecha'])

# Convertir la zona horaria a UTC
df_ESPN['Fecha'] = df_ESPN['Fecha'].dt.tz_localize('UTC')


In [372]:
df_FootballData = df_FootballData.rename(columns={'FootballData_HomeTeam': 'HomeTeam',"FootballData_FTHG":"HomeGoals",
                                                  'FootballData_AwayTeam': 'AwayTeam', "FootballData_FTAG":"AwayGoals",
                                                  'FootballData_Date':"Date"})

df_sofascore = df_sofascore.rename(columns={'Sofascore_team_home': 'HomeTeam',"Sofascore_home_score":"HomeGoals",
                                            'Sofascore_team_away': 'AwayTeam', "Sofascore_away_score":"AwayGoals"})

df_ESPN = df_ESPN.rename(columns={'ESPN_home_team': 'HomeTeam','ESPN_away_team': 'AwayTeam',
                                  "Fecha":"Date" })

In [376]:
prueba = df_sofascore.merge(df_FootballData, on = ["HomeTeam","AwayTeam","HomeGoals","AwayGoals"], how = "outer")
prueba.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57015 entries, 0 to 57014
Columns: 226 entries, Sofascore_number_of_match to FootballData_SYA
dtypes: datetime64[ns, UTC](1), float64(218), object(7)
memory usage: 98.3+ MB


In [385]:
prueba = df_ESPN.merge(df_FootballData, on = ["HomeTeam","AwayTeam","Date"], how = "outer")
prueba.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79211 entries, 0 to 79210
Columns: 238 entries, ESPN_home_league to FootballData_SYA
dtypes: datetime64[ns, UTC](1), float64(153), object(84)
memory usage: 143.8+ MB
